## PET from US LCI (foreground)
 * Build fragment from US LCI foreground
 * encapsulate in an enclosing fragment
 * terminate IO flows to ecoinvent background (since we don't have matrix inversion yet)
 
The purpose of this exercise is to develop the model-building software, so everything should be done 
with interactive scripts.

In [1]:
import re
import os
import sys
import json

sys.path.append('../..')

import lcatools

In [204]:
from lcatools.foreground.fragment_flows import LcFragment
from lcatools.entities import LcFlow, LcQuantity
from lcatools.exchanges import Exchange, comp_dir
from lcatools.catalog import ExchangeRef

In [3]:
F = lcatools.ForegroundManager('pet')

ForegroundArchive archive added in position 0
IlcdLcia archive added in position 1
JSON archive added in position 2
JSON archive added in position 3
JSON archive added in position 4
JSON archive added in position 5
JSON archive added in position 6
JSON archive added in position 7
JSON archive added in position 8
JSON archive added in position 9
JSON archive added in position 10
Setup Catalog and FlowDB... (0.84 s)
Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/uslci_ecospold.json.gz:
Found Extension: zip
19 new quantity entities added (19 total)
4176 new flow entities added (4176 total)
701 new process entities added (701 total)
Fragments loaded... (2.55 s)
finished... (2.55 s)


In [4]:
F.show_all()


LCA Catalog with the following archives:
X [ 0] FG     : ForegroundArchive with 167 entities at pet
  [ 1] ELCD   : /data/LCI/ELCD/ELCD3.2.zip
  [ 2] LCIA   : /data/GitHub/lca-tools-datafiles/catalogs/elcd_lcia_all.json.gz
  [ 3] EI-LCIA: /data/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz
  [ 4] EI-U   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_undefined_xlsx.json.gz
  [ 5] EI_A   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_apos_xlsx.json.gz
  [ 6] EI_Q   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_consequential_xlsx.json.gz
  [ 7] EI_C   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_cut-off_xlsx.json.gz
X [ 8] USLCI  : EcospoldV1Archive with 4896 entities at /home/b/Dropbox/data/USLCI/USLCI_Processes_ecospold1.zip
  [ 9] GaBi-P : /data/GitHub/lca-tools-datafiles/catalogs/gabi_2016_professional-database-2016.json.gz
  [10] GaBi-X : /data/GitHub/lca-tools-datafiles/catalogs/gabi_2016_all-extensions.json.gz


In [5]:
[str(q) for q in F[0].lcia_methods()]

['ILCD2011; Eutrophication freshwater; midpoint; P equivalents; ReCiPe [LCIA]',
 'ILCD2011; Eutrophication marine; midpoint; N equivalents; ReCiPe [LCIA]',
 'ILCD2011; Eutrophication terrestrial; midpoint; Accumulated Exceedance; Seppala et al 2006, Posch et al 2008 [LCIA]',
 'ILCD2011; Respiratory inorganics; midpoint; PM2.5eq; Rabl and Spadaro 2004-Greco et al 2007 [LCIA]',
 'ILCD2011; Photochemical ozone formation; midpoint - human health; POCP; Van Zelm et al. (2008) [LCIA]',
 'ILCD2011; Climate change; midpoint; GWP100; IPCC2007 [LCIA]',
 'ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]',
 'ILCD2011; Land use; midpoint; SOM loss; Mila i Canals et al. (2007) [LCIA]']

In [6]:
F.load_lcia_cfs('LCIA')

Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/elcd_lcia_all.json.gz:
Found Extension: zip
55 new quantity entities added (55 total)
33902 new flow entities added (33902 total)
0 new process entities added (0 total)
0 unmatched flows found from source LCIA... 



In [7]:
['%s %-70.70s' % (x.get_uuid(), x) for x in F['LCIA'].lcia_methods() if re.search('midpoint', x['Name'])]

['40054970-5936-477c-9bec-29fa23b1cb77 ILCD2011; Eutrophication terrestrial; midpoint; Accumulated Exceedance',
 '8b723200-3d8f-4eec-9e10-444edb029476 ILCD2011; Ozone depletion; midpoint; ODP; WMO1999 [LCIA]              ',
 '370960f4-0a3a-415d-bf3e-e5ce63160bb9 ILCD2011; Climate change; midpoint; GWP100; IPCC2007 [LCIA]           ',
 '5296e2be-060b-4e50-b033-d45f85f6ac92 ILCD2011; Eutrophication marine; midpoint; N equivalents; ReCiPe [LCIA',
 'a6f4cbc8-5388-44d6-9a67-53d05684f93e ILCD2011; Ionising radiation; midpoint - human health; ionising radiat',
 'f4602cba-f746-4351-b4da-e91181853e33 ILCD2011; Land use; midpoint; SOM loss; Mila i Canals et al. (2007) [L',
 'f6cbd466-253f-4145-a4bb-8dae7d266e89 ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumula',
 'b7d61a6f-cb2f-4a46-b511-39c3e4cc31d3 ILCD2011; Resource depletion- mineral, fossils and renewables; midpoin',
 'b49c662d-4662-4944-a80a-093dd03d3f4f ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]  

In [82]:
ilcd_qs = ('ec7836be-83eb-41da-bcda-1a6a3fe2d149', 'f6cbd466-253f-4145-a4bb-8dae7d266e89', 
           'f4602cba-f746-4351-b4da-e91181853e33', 'a4630d5b-df2f-48ae-b3f2-5e5257606d68',
           '72ce3013-f678-4c6a-935b-f4752ff8901b', '370960f4-0a3a-415d-bf3e-e5ce63160bb9',
           '5296e2be-060b-4e50-b033-d45f85f6ac92', '40054970-5936-477c-9bec-29fa23b1cb77'
          )

In [83]:
for q in ilcd_qs:
    F.add_to_foreground(F['LCIA'][q])

Add to foreground: ILCD2011; Photochemical ozone formation; midpoint - human health; POCP; Van Zelm et al. (2008) [LCIA]
Add to foreground: ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]
Add to foreground: ILCD2011; Land use; midpoint; SOM loss; Mila i Canals et al. (2007) [LCIA]
Add to foreground: ILCD2011; Eutrophication freshwater; midpoint; P equivalents; ReCiPe [LCIA]
Add to foreground: ILCD2011; Respiratory inorganics; midpoint; PM2.5eq; Rabl and Spadaro 2004-Greco et al 2007 [LCIA]
Add to foreground: ILCD2011; Climate change; midpoint; GWP100; IPCC2007 [LCIA]
Add to foreground: ILCD2011; Eutrophication marine; midpoint; N equivalents; ReCiPe [LCIA]
Add to foreground: ILCD2011; Eutrophication terrestrial; midpoint; Accumulated Exceedance; Seppala et al 2006, Posch et al 2008 [LCIA]


In [6]:
F.load_lcia_cfs('EI-LCIA')

Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz:
**Upstream reference encountered: /data/LCI/Ecoinvent/3.2/undefined

710 new quantity entities added (710 total)
3255 new flow entities added (3255 total)
0 new process entities added (0 total)
92 unmatched flows found from source EI-LCIA... 



In [7]:
F.db.factors_for_quantity('f6cbd466-253f-4145-a4bb-8dae7d266e89')

f6cbd466-253f-4145-a4bb-8dae7d266e89
Characterization Factors
 
CAS Number   |   C0   |   C1   |   C2   |   C3    Flowable
----------------------------------------------------------
007664-41-7      3.02   --         3.02     3.02  Ammonia
007446-09-5      1.31   {{==}}    1.31     1.31  Sulfur dioxide
          .      1.31              1.31     1.31  
007446-11-9      1.05   --         1.05     1.05  Sulfur trioxide
       None      3.02   --       --       --      ammonia
       None    --       --       --         3.02  ammonia
       None    --       --         3.02   --      ammonia
010102-44-0      0.74   {{==}}    0.74     0.74  nitrogen dioxide
010102-43-9      1.13   --         1.13     1.13  nitrogen monoxide
CAS Number   |   C0   |   C1   |   C2   |   C3    Flowable

Columns:
C0: Emissions to air
C1: Emissions to air, unspecified (long-term)
C2: Emissions to non-urban air or from high stacks
C3: Emissions to urban air close to ground


In [8]:
F.load(8)

Archive already loaded; specify reload=True to rewrite


In [9]:
from lcatools.interact import pick_one, cyoa, ifinput, pick_list, _pick_list

In [10]:
def find_flow(name, index=None, elementary=False):
    if name is None:
        name = input('Enter flow name search string: ')
    res = F.search(index, 'flow', Name=name)
    if elementary is not None:
        res = list(filter(lambda x: F.db.is_elementary(x.entity()) == elementary, res))
    pick = pick_one(res)
    print('Picked: %s' % pick)
    return pick
    

In [203]:
current_archive = None

def select_archive():
    F.show_all()
    ch1 = input('Search which catalog? or blank to search all loaded catalogs')
    if len(ch1) == 0:
        index = None
    else:
        try:
            index = int(ch1)
        except ValueError:
            index = F._catalog.get_index(ch1)
    current_archive = index
    return index

def create_fragment(parent=None):
    ch = cyoa('(N)ew flow or (S)earch for flow? ', 'ns')
    if ch == 'n':
        flow = F.new_flow()
    elif ch == 's':
        index = current_archive or select_archive()
        elem = {'i': False,
               'e': True, 
               'a': None}[cyoa('(I)ntermediate, (E)lementary, or (A)ll flows? ', 'aei', 'I').lower()]
        flow = find_flow(None, index=index, elementary=elem)
        if flow is None:
            return None
    else:
        raise ValueError
    direction = { 'i': 'Input', 'o': 'Output'}[cyoa('flow is (I)nput or (O)utput?', 'IO').lower()]
    comment = ifinput('Enter comment: ', '')
    if parent is None:
        # direction reversed for UX! user inputs direction w.r.t. fragment, not w.r.t. parent
        frag = F.new_fragment(flow, comp_dir(direction), Comment=comment)
    else:
        frag = F[0].add_child_fragment_flow(parent, flow.entity(), direction, Comment=comment)
    return frag
    

In [11]:
current_archive = select_archive()


LCA Catalog with the following archives:
X [ 0] FG     : ForegroundArchive with 167 entities at pet
  [ 1] ELCD   : /data/LCI/ELCD/ELCD3.2.zip
X [ 2] LCIA   : IlcdLcia with 1310 entities at /home/b/Dropbox/data/ELCD/ELCD3.2.zip
  [ 3] EI-LCIA: /data/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz
  [ 4] EI-U   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_undefined_xlsx.json.gz
  [ 5] EI_A   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_apos_xlsx.json.gz
  [ 6] EI_Q   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_consequential_xlsx.json.gz
  [ 7] EI_C   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_cut-off_xlsx.json.gz
X [ 8] USLCI  : EcospoldV1Archive with 4896 entities at /home/b/Dropbox/data/USLCI/USLCI_Processes_ecospold1.zip
  [ 9] GaBi-P : /data/GitHub/lca-tools-datafiles/catalogs/gabi_2016_professional-database-2016.json.gz
  [10] GaBi-X : /data/GitHub/lca-tools-datafiles/catalogs/gabi_2016_all-extensions.json.gz
Search which 

In [12]:
frag = create_fragment()

KeyboardInterrupt: 

In [13]:
def find_termination(ref, index=None, direction=None):
    if isinstance(ref, LcFragment):
        if index is None:
            index = current_archive or select_archive()
        terms = F._catalog.terminate_fragment(index, ref)
    else:
        if direction is None:
            direction = { 'i': 'Input', 'o': 'Output'}[cyoa('(I)nput or (O)utput?', 'IO').lower()]
        terms = F._catalog.terminate_flow(flow_ref, direction)
    pick = pick_one(terms)
    print('Picked: %s' % pick)
    return pick
    

In [14]:
def add_termination(frag, term, scenario=None):
    if isinstance(term, ExchangeRef):
        frag.term_from_exch(term, scenario=scenario)
    else:
        frag.terminate(term, scenario=scenario)
    F.build_child_flows(frag, background_children=True)


In [15]:
def auto_terminate(frag, index=None):
    ex = find_termination(frag, index=index)
    add_termination(frag, ex)

In [16]:
def foreground(frag, index=None):
    F.fragment_to_foreground(frag)
    auto_terminate(frag)

In [15]:
auto_terminate(frag)

(selecting only choice)
Picked: (USLCI) Polyethylene terephthalate, resin, at plant [Chemical Manufacturing, Petrochemical Manufacturing] <- Polyethylene terephthalate, resin, at plant [RNA]
LcFragment - Name: Acetic acid, at plant:
BG: ((B) ref) -<- d002717 -<- ---:  Acetic acid, at plant
created null termination in background
LcFragment - Name: CUTOFF Disposal, solid waste, unspecified, to municipal incineration:
BG: ((B) ref) -<- a628321 -<- ---:  CUTOFF Disposal, solid waste, unspecified, to municipal incineration
created null termination in background
LcFragment - Name: Methanol, at plant:
BG: ((B) ref) -<- f892bf8 -<- ---:  Methanol, at plant
created null termination in background
LcFragment - Name: Transport, pipeline, natural gas:
BG: ((B) ref) -<- 1b76dbd -<- ---:  Transport, pipeline, natural gas
created null termination in background
LcFragment - Name: CUTOFF Ethylene glycol, at plant:
BG: ((B) ref) -<- f83a957 -<- ---:  CUTOFF Ethylene glycol, at plant
created null terminat

In [208]:
def del_orphans(self, for_real=False):
    """
    self is a foreground archive
    """
    for f in self.fragments(show_all=True):
        if f.reference_entity is not None:
            continue
        try:
            next(_find_links(self, f))
            print('Found a link for %s' % f)
        except StopIteration:
            print('Deleting %s' % f)
            if for_real:
                del self._entities[f._uuid]


def _find_links(self, frag):
    for i in self.fragments(show_all=True):
        if i.reference_entity is frag:
            yield i
        else:
            for v in i._terminations.values():
                if v.term_node is frag:
                    yield i
    

In [211]:
del_orphans(F[0])

Found a link for ( ** ref) -<- d002717 -<- -*    Acetic acid, at plant
Found a link for ( ** ref) -<- 410a96e -<- -O    something that requires gas transport
Found a link for ( ** ref) -<- cd48c3a -<- -*    Transport, barge, diesel powered
Found a link for ( ** ref) -<- e7ace8c -<- -*    Transport, train, diesel powered
Found a link for ( ** ref) -<- 2c8ba87 -<- -*    Transport, barge, average fuel mix
Found a link for ( ** ref) -<- 114c1d5 -<- -*    Transport, combination truck, diesel powered
Found a link for ( ** ref) -<- 0a4588e -<- -*    Transport, barge, residual fuel oil powered
Found a link for ( ** ref) -<- 1b76dbd -<- -*    Transport, pipeline, natural gas
Found a link for ( ** ref) -<- 68d264d -<- -*    Transport, combination truck, average fuel mix
Found a link for ( ** ref) -<- f2118f0 -<- -*    Transport, pipeline, unspecified petroleum products
Found a link for ( ** ref) -<- 0c44dc5 -<- -*    Polyethylene terephthalate, resin, at plant
Found a link for ((B) ref) -<- 5dfd

In [12]:
F.draw_fragment(F.frag('1b7'))

-<--*   1b76dbd (      1) Transport, pipeline, natural gas
    | -<--*   b78ac6f (  0.012) Natural gas, combusted in industrial equipment
    |     | -<----: 9ca8b7d (    1.2) Transport, pipeline, natural gas
    |     | -<--#   78a3212 (    0.2) Transport, combination truck, average fuel mix
    |     | -<--#   dc450e8 (  0.012) Transport, train, diesel powered
    |     | -<---C  e038531 (      1) Natural gas, processed, at plant
    |     x 
    x 


In [54]:
F.frag('b14').terminate(None)

In [30]:
[str(x) for x in F[0].fragments(show_all=True, background=False, match='combust')]

['(0c44dc5) -<- 4f772f2 -<- -*    Residual fuel oil, combusted in industrial boiler',
 '(0c44dc5) -<- 1a7d38d -<- -*    Natural gas, combusted in industrial boiler',
 '(0c44dc5) -<- a95908c -<- -*    Bituminous coal, combusted in industrial boiler',
 '(1b76dbd) -<- b78ac6f -<- -*    Natural gas, combusted in industrial equipment',
 '(0c44dc5) -<- 4083ab8 -<- -*    Diesel, combusted in industrial equipment']

In [18]:
F.show_fragments(show_all=False, background=False)

( ** ref) -<- d002717 -<- -*    Acetic acid, at plant
( ** ref) -<- cd48c3a -<- -*    Transport, barge, diesel powered
( ** ref) -<- e7ace8c -<- -*    Transport, train, diesel powered
( ** ref) -<- 2c8ba87 -<- -*    Transport, barge, average fuel mix
( ** ref) -<- 114c1d5 -<- -*    Transport, combination truck, diesel powered
( ** ref) -<- 0a4588e -<- -*    Transport, barge, residual fuel oil powered
( ** ref) -<- 1b76dbd -<- -*    Transport, pipeline, natural gas
( ** ref) -<- 68d264d -<- -*    Transport, combination truck, average fuel mix
( ** ref) -<- f2118f0 -<- -*    Transport, pipeline, unspecified petroleum products
( ** ref) -<- 0c44dc5 -<- -*    Polyethylene terephthalate, resin, at plant


In [13]:
F.compute_unit_scores()

In [14]:
F.show_all()


LCA Catalog with the following archives:
X [ 0] FG     : ForegroundArchive with 167 entities at pet
  [ 1] ELCD   : /data/LCI/ELCD/ELCD3.2.zip
X [ 2] LCIA   : IlcdLcia with 33957 entities at /home/b/Dropbox/data/ELCD/ELCD3.2-a.zip
  [ 3] EI-LCIA: /data/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz
  [ 4] EI-U   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_undefined_xlsx.json.gz
  [ 5] EI_A   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_apos_xlsx.json.gz
  [ 6] EI_Q   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_consequential_xlsx.json.gz
  [ 7] EI_C   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_cut-off_xlsx.json.gz
X [ 8] USLCI  : EcospoldV1Archive with 4896 entities at /home/b/Dropbox/data/USLCI/USLCI_Processes_ecospold1.zip
  [ 9] GaBi-P : /data/GitHub/lca-tools-datafiles/catalogs/gabi_2016_professional-database-2016.json.gz
  [10] GaBi-X : /data/GitHub/lca-tools-datafiles/catalogs/gabi_2016_all-extensions.json.gz


In [15]:
frag = F.frag('0c4')

In [224]:
_qs = (x for x in F[0].lcia_methods())

In [223]:
results = F.fg_lcia(frag.term.term_node)

In [225]:
q = next(_qs)
str(q)

'ILCD2011; Climate change; midpoint; GWP100; IPCC2007 [LCIA]'

In [242]:
F.db.flowables.synonyms_for('010102-44-0')

{'010102-44-0',
 '08a91e70-3ddc-11dd-96e5-0050c2490048',
 '08a91e70-3ddc-11dd-96e6-0050c2490048',
 '08a91e70-3ddc-11dd-96e7-0050c2490048',
 '08a91e70-3ddc-11dd-96e8-0050c2490048',
 '08a91e70-3ddc-11dd-96e9-0050c2490048',
 '08a91e70-3ddc-11dd-96ea-0050c2490048',
 '08a91e70-3ddc-11dd-96eb-0050c2490048',
 '08a91e70-3ddc-11dd-96ec-0050c2490048',
 '08a91e70-3ddc-11dd-96ed-0050c2490048',
 '29063b80-6556-11dd-ad8b-0800200c9a66',
 '29063b81-6556-11dd-ad8b-0800200c9a66',
 '29063b82-6556-11dd-ad8b-0800200c9a66',
 '2fb4ca68-6eb1-3f0b-b907-70cdc6f050f2',
 'AC1MHUB8',
 'Dinitrogen tetroxide',
 'NITROGEN TETROXIDE',
 'ac1mhub8',
 'dinitrogen tetroxide',
 'nitrogen dioxide',
 'nitrogen tetroxide'}

In [69]:
[x.flow.get_uuid() for x in frag.term.term_node.entity().exchanges() if x.flow['Name'] == 'Nitrogen oxides ']

['2fb4ca68-6eb1-3f0b-b907-70cdc6f050f2']

In [240]:
F.db.flowables.add_synonym('010102-44-0', '2fb4ca68-6eb1-3f0b-b907-70cdc6f050f2')

1839

In [226]:
results[q.get_uuid()].show_details(show_all=True)

ILCD2011; Climate change; midpoint; GWP100; IPCC2007 [LCIA] kg
------------------------------------------------------------
    0.0724 x          1 =     0.0724 [GLO] Carbon dioxide, fossil  [air, unspecified]
   0.00016 x         25 =      0.004 [GLO] Methane  [air, unspecified]
   9.7e-10 x       ---- =          0 [GLO] Aluminum [water, unspecified]
   0.00011 x       ---- =          0 [GLO] Ammonia [water, unspecified]
   5.1e-05 x       ---- =          0 [GLO] Acetic acid [air, unspecified]
   4.4e-05 x       ---- =          0 [GLO] TOC, Total Organic Carbon [water, unspecified]
   0.00568 x       ---- =          0 [GLO] Carbon monoxide [air, unspecified]
     4e-05 x       ---- =          0 [GLO] Methyl acetate [air, unspecified]
    0.0003 x       ---- =          0 [GLO] BOD5, Biological Oxygen Demand [water, unspecified]
   7.9e-05 x       ---- =          0 [GLO] Bromine [air, unspecified]
   9.7e-10 x       ---- =          0 [GLO] Iron [water, unspecified]
   0.00015 x       --

In [227]:
F.db.factors_for_quantity(q)

ILCD2011; Climate change; midpoint; GWP100; IPCC2007 [LCIA]
Characterization Factors
 
CAS Number   |   C0   |   C1   |   C2   |   C3   |   C4   |   C5   |   C6   |   C7   |   C8   |   C9    Flowable
----------------------------------------------------------------------------------------------------------------
000431-89-0  3.22e+03 3.22e+03   --     3.22e+03 3.22e+03   --       --       --       --       --      1,1,1,2,3,3,3-heptafluoropropane
000431-63-0  1.37e+03 1.37e+03   --     1.37e+03 1.37e+03   --       --       --       --       --      1,1,1,2,3,3-hexafluoropropane
000378-16-5        11       11   --           11       11   --       --       --       --       --      2,2,3,3,3-Pentafluoropropyl methyl ether
000076-15-3  7.37e+03 7.37e+03 7.37e+03 7.37e+03 7.37e+03   --       --       --       --       --      CFC-115
000075-72-9  1.44e+04 1.44e+04 1.44e+04 1.44e+04 1.44e+04   --       --       --       --       --      CFC-13
000420-46-2  4.47e+03 4.47e+03   --     4.47e+03

In [233]:
for fb in F.db.flowables.search('chromium III'):
    print(F.db.flowables.synonym_set(fb))

{'016065-83-1', 'cr+3', 'cr3+', 'CR+3', '08a91e70-3ddc-11dd-9f79-0050c2490048', 'chromium, ion (cr3+)', 'chromium (iii) ion', 'Cr3+', 'Chromic ion', 'chromium(3+)ion', 'Chromium(III) compounds, n.o.s', 'chromium(iii), chromium(iii) insoluble salts, chromic ion, chromium ion, trivalent chromium', 'chromium iii', 'Chromium, ion (Cr 3+)', 'chromium ion(cr3+)', 'Chromium (III), insoluble salts', 'chromium(3+)', 'Cr(3+)', 'Chromium, ion(Cr3+)', 'chromium(3+) ions', '4d9a8790-3ddd-11dd-8fd2-0050c2490048', '4d9a8790-3ddd-11dd-8fd1-0050c2490048', 'Chromium (III) ion', 'chromium, ion(cr3+)', '08a91e70-3ddc-11dd-9f7c-0050c2490048', 'Chromium (3+)', 'Chromium(III), Chromium(III) insoluble salts, Chromic ion, Chromium ion, trivalent Chromium', 'chromium (iii), insoluble salts', 'Chromium(III) compounds, n.o.s.', 'ac1l1dpd', '4d9a8790-3ddd-11dd-8fd0-0050c2490048', 'Chromium ion (3+)', '4d9a8790-3ddd-11dd-8fd4-0050c2490048', 'CHROMIUM (III)', 'Chromium ion(Cr3+)', '3e4c8d2d-6556-11dd-ad8b-0800200c9a

In [234]:
for fb in F.db.flowables.search('chromium VI'):
    print(F.db.flowables.synonym_set(fb))

{'Chromium(VI) compounds', 'chromium (hexavalent compounds)', 'chromium VI', 'chromium compounds, hexavalent', 'Chromium(VI) compounds, n.o.s', 'fe0acd60-3ddc-11dd-a70d-0050c2490048', '018540-29-9', 'CR+6', '08a91e70-3ddc-11dd-950b-0050c2490048', 'Chromium(6+)', 'chromium(6+)ion', 'Chromium, hexavalent', 'Chromium VI', '4d9a8790-3ddd-11dd-8fd7-0050c2490048', '4d9a8790-3ddd-11dd-8fd5-0050c2490048', '3e4c8d50-6556-11dd-ad8b-0800200c9a66', 'chromium(VI) cation', 'fe0acd60-3ddc-11dd-a70c-0050c2490048', 'cr(6+)', 'Chromium, ion (Cr6+)', 'Chromium, ion (Cr 6+)', 'Chromium(6+)ion', 'chromium, ion(cr6+)', 'chromium(vi)', '4d9a8790-3ddd-11dd-9c1f-0050c2490048', 'Chromium(6+)ions', 'hexavalent chromium', 'chromium (cr(sup 6+))', 'Cr6+', 'ac1l1gta', 'Chromium (hexavalent compounds)', 'chromium(vi) ions', 'NCI-C04273', 'Chromium compounds, hexavalent', 'AC1L1GTA', '(51cr)chromium(6+)', '4d9a8790-3ddd-11dd-8fd6-0050c2490048', '4d9a8790-3ddd-11dd-8fd8-0050c2490048', 'fe0acd60-3ddc-11dd-a70b-0050c249

In [111]:
F.show_fragments()

( ** ref) -<- d002717 -<- -*    Acetic acid, at plant
( ** ref) -<- cd48c3a -<- -*    Transport, barge, diesel powered
( ** ref) -<- e7ace8c -<- -*    Transport, train, diesel powered
( ** ref) -<- 2c8ba87 -<- -*    Transport, barge, average fuel mix
( ** ref) -<- 114c1d5 -<- -*    Transport, combination truck, diesel powered
( ** ref) -<- 0a4588e -<- -*    Transport, barge, residual fuel oil powered
( ** ref) -<- 1b76dbd -<- -*    Transport, pipeline, natural gas
( ** ref) -<- 68d264d -<- -*    Transport, combination truck, average fuel mix
( ** ref) -<- f2118f0 -<- -*    Transport, pipeline, unspecified petroleum products
( ** ref) -<- 0c44dc5 -<- -*    Polyethylene terephthalate, resin, at plant
((B) ref) -<- 5dfdb2f -<- ---:  Bituminous coal, at mine
((B) ref) -<- 0f4d896 -<- ---:  CUTOFF Transport, pipeline, coal slurry
((B) ref) -<- a8a990e -<- ---:  CUTOFF Disposal, solid waste, unspecified, to unspecified treatment
((B) ref) -<- 6f3a5da -<- ---:  CUTOFF Disposal, solid waste, u

In [95]:
F.frag('1b7').direction

'Input'

In [80]:
ios = F.get_fragment_inventory(F.frag('1b7'))

descending
descending
descending


In [144]:
ffs, _ = F.frag('1b7').traverse(lambda x: F.child_flows(x), 1.0, None)

descending
descending
descending


In [145]:
ffs

In [151]:
F.draw_fragment(F.frag('1b7'))

-<--*   1b76dbd (      1) Transport, pipeline, natural gas
    | -<--*   b78ac6f (  0.012) Natural gas, combusted in industrial equipment
    |     | -<----: 9ca8b7d (    1.2) Transport, pipeline, natural gas
    |     | -<--#   78a3212 (    0.2) Transport, combination truck, average fuel mix
    |     | -<--#   dc450e8 (  0.012) Transport, train, diesel powered
    |     | -<---C  e038531 (      1) Natural gas, processed, at plant
    |     x 
    x 


In [158]:
F.draw_fragment(F.frag('78a').term.term_node)

-<--*   68d264d (      1) Transport, combination truck, average fuel mix
    | -<--#   cf3ed88 (      1) Transport, combination truck, diesel powered
    x 


In [159]:
F.draw_fragment(F.frag('cf3').term.term_node)

-<--*   114c1d5 (      1) Transport, combination truck, diesel powered
    | -<---C  4b85d4b (  0.027) Diesel, at refinery
    x 


In [174]:
tot = 0.0
for f in ffs:
    if f.term.score_cache(q) is None:
        score = '--'
        total = ' '
    else:
        scr = f.term.score_cache(q).total()
        score = '%-10.3g' % scr
        total = '%-10.3g' % (scr * f.node_weight)
        tot += (scr * f.node_weight)
    print('%-10.10s %-10.10s %s' % (total, score, f))
print('%-10.3g Total' % tot)
            

0          0          1b76dbd          1 [ Input] -#  Transport, pipeline, natural gas
1.51e-05   0.00125    b78ac6f     0.0121 [ Input] -#  Natural gas, combusted in industrial equipment
           --         78a3212    0.00241 [ Input] -#  Transport, combination truck, average fuel mix
0          0          68d264d    0.00241 [ Input] -#  Transport, combination truck, average fuel mix
           --         cf3ed88    0.00241 [ Input] -#  Transport, combination truck, diesel powered
1e-06      0.000417   114c1d5    0.00241 [ Input] -#  Transport, combination truck, diesel powered
           --         4b85d4b   6.56e-05 [ Input] --: Diesel, at refinery
           --         9ca8b7d     0.0144 [ Input] --: Transport, pipeline, natural gas
           --         e038531     0.0121 [ Input] --: Natural gas, processed, at plant
           --         dc450e8   0.000144 [ Input] -#  Transport, train, diesel powered
5.39e-08   0.000374   e7ace8c   0.000144 [ Input] -#  Transport, train, diese

In [183]:
['%10.6g %s' % (x.value, x.flow) for x in F.db.filter_exch(F.frag('b78').term.term_node, elem=False)]

['    0.1985 Transport, combination truck, average fuel mix [Truck Transportation, General Freight Trucking]',
 '         1 Natural gas, processed, at plant [Chemical Manufacturing, Petrochemical Manufacturing]',
 '  0.011858 Transport, train, diesel powered [Rail Transportation, Rail Transportation]',
 '     1.186 Transport, pipeline, natural gas [Utilities, Fossil Fuel Electric Power Generation]',
 '         1 Natural gas, combusted in industrial equipment [Utilities, Steam and Air-Conditioning Supply]']

In [184]:
['%10.6g %s' % (x.value, x.flow) for x in F.db.filter_exch(F.frag('1b7').term.term_node, elem=False)]

[' 0.0121378 Natural gas, combusted in industrial equipment [Utilities, Steam and Air-Conditioning Supply]',
 '         1 Transport, pipeline, natural gas [Utilities, Fossil Fuel Electric Power Generation]']

In [220]:
(.0121378 )/ (1.0 - .0144)

0.012315137987012987

In [191]:
F.frag('1b7').term.term_flow.origin

'/home/b/Dropbox/data/USLCI/USLCI_Processes_ecospold1.zip'

In [157]:
ffs[1].__dict__

{'fragment': <lcatools.foreground.fragment_flows.LcFragment at 0x7fbfcc92b860>,
 'is_conserved': False,
 'magnitude': 0.0121378329004437,
 'node_weight': 0.0121378329004437,
 'term': <lcatools.foreground.fragment_flows.FlowTermination at 0x7fbfc9cdfa90>}

In [156]:
F.elementary(ffs[0].term.term_node)

Elementary exchanges:
Output:     [0.000138 [kg]] Carbon dioxide, fossil  [air, unspecified]
Output:     [0.00457 [kg]] Methane, fossil [air, unspecified]


In [192]:
fnew = create_fragment()

(N)ew flow or (S)earch for flow? N
Enter flow name: something that requires gas transport
Enter CAS number (or none):  []: 
Choose reference quantity: 

Choice Item
====== ======================================================================
 [0]   (30) (none)
 [1]   (4) ILCD2011
 [2]   (4) ILCD2011; ReCiPe2008
  (D)  done (keep all)
------ ----------------------------------------------------------------------
Grouped by Method
Enter choice (or "None"): 0
(0, None)
(selecting only choice (none))

Select item: 

Choice Item
====== ======================================================================
 [ 0]  Area
 [ 1]  Area*time
 [ 2]  Carbon content (biogenic)
 [ 3]  Carbon content (fossil)
 [ 4]  Duration
 [ 5]  EcoSpold Quantity MJ
 [ 6]  EcoSpold Quantity kWh
 [ 7]  EcoSpold Quantity kg
 [ 8]  EcoSpold Quantity l
 [ 9]  EcoSpold Quantity m3
 [10]  EcoSpold Quantity t*km
 [11]  Energy from renewable raw materials (net cal. value)
 [12]  Energy renewable and non renewable (gross calo

In [198]:
F.terminate_to_foreground(fnew)

In [212]:
fnew2 = create_fragment(fnew)

(N)ew flow or (S)earch for flow? s

LCA Catalog with the following archives:
X [ 0] FG     : ForegroundArchive with 169 entities at pet
  [ 1] ELCD   : /data/LCI/ELCD/ELCD3.2.zip
X [ 2] LCIA   : IlcdLcia with 33957 entities at /home/b/Dropbox/data/ELCD/ELCD3.2-a.zip
  [ 3] EI-LCIA: /data/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz
  [ 4] EI-U   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_undefined_xlsx.json.gz
  [ 5] EI_A   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_apos_xlsx.json.gz
  [ 6] EI_Q   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_consequential_xlsx.json.gz
  [ 7] EI_C   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_cut-off_xlsx.json.gz
X [ 8] USLCI  : EcospoldV1Archive with 4896 entities at /home/b/Dropbox/data/USLCI/USLCI_Processes_ecospold1.zip
  [ 9] GaBi-P : /data/GitHub/lca-tools-datafiles/catalogs/gabi_2016_professional-database-2016.json.gz
  [10] GaBi-X : /data/GitHub/lca-tools-datafiles/catalogs/gabi_201

In [216]:
F.draw_fragment(fnew)

-<--O   410a96e (      1) something that requires gas transport
    | -<--#   6c6ec01 (      1) Transport, pipeline, natural gas
    |     | -<---C  b984568 (  0.012) Natural gas, processed, at plant
    |     | -<---C  8b9f144 (6.7e-05) Diesel, at refinery
    |     x 
    x 


In [219]:
F.frag('b98').exchange_value(0)

0.01231511476334841

In [214]:
F.frag('6c6e').terminate(F.frag('1b7'))

In [215]:
F.build_child_flows(F.frag('6c6e'))

descending
descending
descending
LcFragment - Name: Natural gas, processed, at plant:
LcFragment - Name: Diesel, at refinery:


In [146]:
R = traversal_to_lcia(ffs)

In [147]:
lcatools.show_lcia(R)

LCIA Results
------------------------------------------------------------
9.1944e-05 ILCD2011; Eutrophication terrestrial; midpoint; Accumulated Exceedance; Seppala et al 2006, Posch et al 2008 [LCIA]
6.8058e-05 ILCD2011; Photochemical ozone formation; midpoint - human health; POCP; Van Zelm et al. (2008) [LCIA]
8.3958e-06 ILCD2011; Eutrophication marine; midpoint; N equivalents; ReCiPe [LCIA]
   0.13836 ILCD2011; Climate change; midpoint; GWP100; IPCC2007 [LCIA]
 1.612e-07 ILCD2011; Respiratory inorganics; midpoint; PM2.5eq; Rabl and Spadaro 2004-Greco et al 2007 [LCIA]
         0 ILCD2011; Land use; midpoint; SOM loss; Mila i Canals et al. (2007) [LCIA]
1.6189e-05 ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]
         0 ILCD2011; Eutrophication freshwater; midpoint; P equivalents; ReCiPe [LCIA]


In [148]:
R[q.get_uuid()].show_details()

ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA] mol
------------------------------------------------------------
             Total score: 0 
  0.000144 x   0.000374 =   5.39e-08 [RNA] Transport, train, diesel powered [Rail Transportation, Rail Transportation]
             Total score: 5.38937e-08 
    0.0121 x    0.00125 =   1.51e-05 [RNA] Natural gas, combusted in industrial equipment [Utilities, Steam and Air-Conditioning Supply]
             Total score: 1.51304e-05 
   0.00241 x   0.000417 =      1e-06 [RNA] Transport, combination truck, diesel powered [Truck Transportation, General Freight Trucking]
             Total score: 1.00453e-06 
             Total score: 0 
  1.62e-05 ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]


In [149]:
R[q.get_uuid()].show_components()

ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA] mol
------------------------------------------------------------
  1.51e-05 (1b76dbd) -<- b78ac6f -<- -*    Natural gas, combusted in industrial equipment
     1e-06 ( ** ref) -<- 114c1d5 -<- -*    Transport, combination truck, diesel powered
  5.39e-08 ( ** ref) -<- e7ace8c -<- -*    Transport, train, diesel powered
         0 ( ** ref) -<- 68d264d -<- -*    Transport, combination truck, average fuel mix
         0 ( ** ref) -<- 1b76dbd -<- -*    Transport, pipeline, natural gas
  1.62e-05 ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]


In [116]:
str(q)

'ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]'

In [119]:
F.draw_fragment(F.frag('1b7'))

-<--*   1b76dbd (      1) Transport, pipeline, natural gas
    | -<--*   b78ac6f (  0.012) Natural gas, combusted in industrial equipment
    |     | -<----: 9ca8b7d (    1.2) Transport, pipeline, natural gas
    |     | -<--#   78a3212 (    0.2) Transport, combination truck, average fuel mix
    |     | -<--#   dc450e8 (  0.012) Transport, train, diesel powered
    |     | -<---C  e038531 (      1) Natural gas, processed, at plant
    |     x 
    x 


In [140]:
R[q.get_uuid()].show_components()

ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA] mol
------------------------------------------------------------
  1.51e-05 (1b76dbd) -<- b78ac6f -<- -*    Natural gas, combusted in industrial equipment
     1e-06 ( ** ref) -<- 114c1d5 -<- -*    Transport, combination truck, diesel powered
  5.39e-08 ( ** ref) -<- e7ace8c -<- -*    Transport, train, diesel powered
         0 ( ** ref) -<- 68d264d -<- -*    Transport, combination truck, average fuel mix
         0 ( ** ref) -<- 1b76dbd -<- -*    Transport, pipeline, natural gas
  1.62e-05 ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]


In [120]:
F.draw_fragment(F.frag('68d'))

-<--*   68d264d (      1) Transport, combination truck, average fuel mix
    | -<--#   cf3ed88 (      1) Transport, combination truck, diesel powered
    x 


In [136]:
R = F.fg_lcia(F.frag('114').term.term_node)

In [137]:
R[q.get_uuid()].show_details()

ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA] mol
------------------------------------------------------------
  0.000532 x       0.74 =   0.000394 [GLO] Nitrogen oxides  [air, unspecified]
  1.76e-05 x       1.31 =    2.3e-05 [GLO] Sulfur oxides [air, unspecified]
             Total score: 0.00041693 
  0.000417 ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]


In [132]:
for f in F[0].fragments(show_all=True):
    f.term.clear_score_cache()

In [133]:
%time F.compute_unit_scores()

All characterizations have the same value- picking first one
All characterizations have the same value- picking first one
All characterizations have the same value- picking first one
CPU times: user 200 ms, sys: 0 ns, total: 200 ms
Wall time: 200 ms


In [134]:
F.fragment_lcia(F.frag('114'))[q.get_uuid()].show_details()

ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA] mol
------------------------------------------------------------
         1 x   0.000417 =   0.000417 [RNA] Transport, combination truck, diesel powered [Truck Transportation, General Freight Trucking]
             Total score: 0.00041693 
  0.000417 ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]


In [124]:
from lcatools.foreground.fragment_flows import traversal_to_lcia

In [125]:
F.show_fragments()

( ** ref) -<- d002717 -<- -*    Acetic acid, at plant
( ** ref) -<- cd48c3a -<- -*    Transport, barge, diesel powered
( ** ref) -<- e7ace8c -<- -*    Transport, train, diesel powered
( ** ref) -<- 2c8ba87 -<- -*    Transport, barge, average fuel mix
( ** ref) -<- 114c1d5 -<- -*    Transport, combination truck, diesel powered
( ** ref) -<- 0a4588e -<- -*    Transport, barge, residual fuel oil powered
( ** ref) -<- 1b76dbd -<- -*    Transport, pipeline, natural gas
( ** ref) -<- 68d264d -<- -*    Transport, combination truck, average fuel mix
( ** ref) -<- f2118f0 -<- -*    Transport, pipeline, unspecified petroleum products
( ** ref) -<- 0c44dc5 -<- -*    Polyethylene terephthalate, resin, at plant
((B) ref) -<- 5dfdb2f -<- ---:  Bituminous coal, at mine
((B) ref) -<- 0f4d896 -<- ---:  CUTOFF Transport, pipeline, coal slurry
((B) ref) -<- a8a990e -<- ---:  CUTOFF Disposal, solid waste, unspecified, to unspecified treatment
((B) ref) -<- 6f3a5da -<- ---:  CUTOFF Disposal, solid waste, u

In [94]:
[str(x) for x in ffs]

['4b85d4b   6.56e-05 [ Input] --: Diesel, at refinery',
 '9ca8b7d     0.0144 [ Input] --: Transport, pipeline, natural gas',
 'e038531     0.0121 [ Input] --: Natural gas, processed, at plant',
 '0183c1a   9.33e-07 [ Input] --: Diesel, at refinery']

In [109]:
['%6.3g %s %s' % (z.value, z.direction, z.flow) for z in ios]

['0.0123 Input Natural gas, processed, at plant [Chemical Manufacturing, Petrochemical Manufacturing]',
 '6.75e-05 Input Diesel, at refinery [Petroleum and Coal Products Manufacturing, Petroleum Refineries]']

In [217]:
(6.56e-5 + 9.33e-7) / (1 - 0)

6.6533e-05

In [105]:
ios[1].value

6.749702035026239e-05

In [ ]:
f.

In [53]:
F.find_flowable('Nitrogen oxides     '.strip())

{16}

In [47]:
fb

set()

In [41]:
cm = F.db.find_matching_compartment(['Emissions to air'])

In [22]:
F.draw_fragment(F.frag('0c4'))

-<--*   0c44dc5 (      1) Polyethylene terephthalate, resin, at plant
    | -<----: db326b6 (   0.32) CUTOFF Ethylene glycol, at plant
    | -<----: 123d445 (   0.52) Paraxylene, at plant
    | -<--#   7771ba3 (    1.6) Transport, train, diesel powered
    | -<--*   a95908c (  0.019) Bituminous coal, combusted in industrial boiler
    |     | -<--#   c1f3eea (      1) Transport, train, diesel powered
    |     | -<--#   7dde672 ( 0.0068) Transport, combination truck, diesel powered
    |     | -<--#   f578071 (   0.13) Transport, barge, average fuel mix
    |     | -<---C  630f118 (      1) Bituminous coal, at mine
    |     | -<---C  2728068 (  0.079) CUTOFF Disposal, ash and flue gas desulfurization sludge, to unspecified reuse
    |     | -<---C  a4252ab (  0.005) CUTOFF Transport, pipeline, coal slurry
    |     | -<---C  c4b018b (  0.062) CUTOFF Disposal, solid waste, unspecified, to unspecified treatment
    |     x 
    | -<--*   4f772f2 (  0.027) Residual fuel oil, combusted in

In [21]:
lcatools.show_lcia(F.frag('0c4').term._score_cache)

LCIA Results
------------------------------------------------------------
         0 ILCD2011; Eutrophication freshwater; midpoint; P equivalents; ReCiPe [LCIA]
         0 ILCD2011; Climate change; midpoint; GWP100; IPCC2007 [LCIA]
         0 ILCD2011; Respiratory inorganics; midpoint; PM2.5eq; Rabl and Spadaro 2004-Greco et al 2007 [LCIA]
         0 ILCD2011; Eutrophication marine; midpoint; N equivalents; ReCiPe [LCIA]
         0 ILCD2011; Eutrophication terrestrial; midpoint; Accumulated Exceedance; Seppala et al 2006, Posch et al 2008 [LCIA]
         0 ILCD2011; Acidification terrestrial and freshwater; midpoint; Accumulated Exceedance; Seppälä et al. 2006, Posch et al. 2008 [LCIA]
         0 ILCD2011; Land use; midpoint; SOM loss; Mila i Canals et al. (2007) [LCIA]
         0 ILCD2011; Photochemical ozone formation; midpoint - human health; POCP; Van Zelm et al. (2008) [LCIA]


In [55]:
foreground(F.frag('d00'))

(selecting only choice)
Picked: (USLCI) Acetic acid, at plant [Chemical Manufacturing, All Other Basic Organic Chemical Manufacturing] <- Acetic acid, at plant [RNA]
found termination in foreground
LcFragment - Name: Methanol, at plant:
found termination in foreground
LcFragment - Name: Electricity, at cogen, for natural gas turbine:
LcFragment - Name: Electricity, at grid, US, 2000:
BG: ((B) ref) -<- 86c34c7 -<- ---:  Electricity, at grid, US, 2000
created null termination in background
found termination in foreground
found termination in foreground
found termination in foreground
LcFragment - Name: Natural gas, combusted in industrial boiler:
BG: ((B) ref) -<- 0ac9934 -<- ---:  Natural gas, combusted in industrial boiler
created null termination in background
found termination in foreground
LcFragment - Name: Natural gas, processed, at plant:
LcFragment - Name: CUTOFF Disposal, solid waste, unspecified, to sanitary landfill:
LcFragment - Name: Recovered energy, at acetic acid product

In [51]:
F.frag('db3').terminate(None)

In [92]:
F.save()

Saving foreground
Catalog file with 11 archives saved to foreground


In [61]:
for f in F[0].fragments(background=False, show_all=True, match='combus'):
     if not f.term.is_null:
        if f.term.is_bg:
            foreground(f)
            # print('%s' % f)


(selecting only choice)
Picked: (USLCI) Natural gas, combusted in industrial boiler [Utilities, Steam and Air-Conditioning Supply] <- Natural gas, combusted in industrial boiler [RNA]
found termination in foreground
found termination in foreground
found termination in foreground
LcFragment - Name: Natural gas, processed, at plant:


In [73]:
del_orphans(F[0])

Deleting (1f65198) -<- 02514d8 -<- --C   Transport, pipeline, unspecified petroleum products
Deleting (1f65198) -<- 3962b4b -<- --C   Natural gas, processed, at plant
Deleting (47fe10f) =>= 39b326b =>= --C   Recovered energy
Deleting (1f65198) -<- 268cdea -<- -#    Transport, barge, diesel powered
Deleting (96aa479) -<- d90abba -<- ---:  Transport, pipeline, natural gas
Deleting (a38e842) -<- eb7848f -<- -#    Transport, combination truck, diesel powered
Deleting (ba3ce83) -<- 32dd01c -<- --C   Diesel, at refinery
Deleting (96aa479) -<- e6b0be5 -<- -#    Transport, combination truck, average fuel mix
Deleting (47fe10f) -<- 19250a5 -<- --C   Methanol, at plant
Deleting (47fe10f) -<- 4bdce66 -<- --C   CUTOFF Disposal, solid waste, unspecified, to sanitary landfill
Deleting (47fe10f) -<- 8bb6f1e -<- --C   Electricity, at cogen, for natural gas turbine
Deleting (8ce3823) -<- 459b063 -<- --C   Diesel, at refinery
Deleting (47fe10f) -<- ee32fac -<- --C   Natural gas, combusted in industrial 

In [69]:
del F[0]._entities[F.frag('4ec')._uuid]

In [67]:
F[0]._entities.remove(F.frag('4ec'))

AttributeError: 'dict' object has no attribute 'remove'

In [70]:
F.show_fragments(show_all=True, background=True, match='transport')

((B) ref) -<- 5be0f02 -<- ---:  Transport, barge, residual fuel oil powered
((B) ref) -<- fcf437d -<- ---:  Transport, pipeline, unspecified petroleum products
((B) ref) -<- 0fb2f9c -<- ---:  Transport, train, diesel powered
((B) ref) -<- e617086 -<- ---:  Transport, pipeline, natural gas


In [49]:
F.frag('68f').terminate(F.frag('ba3'))

In [40]:
[str(x) for x in frag.io_flows(lambda x: F.child_flows(x), None)]

['561ef05   0.000183 [ Input] --: Transport, combination truck, diesel powered',
 'e756aee     0.0201 [ Input] --: Methanol, at plant',
 '214d2bc   9.86e-05 [ Input] --: Transport, train, diesel powered',
 'ac6c85d    0.00165 [ Input] --: Transport, combination truck, average fuel mix',
 '5174ff7    0.00832 [ Input] --: Natural gas, processed, at plant',
 '071b31a    0.00986 [ Input] --: Transport, pipeline, natural gas',
 '44366ab   2.08e-05 [ Input] --: CUTOFF Disposal, solid waste, unspecified, to sanitary landfill',
 '89983cf    0.00701 [Output] --: Recovered energy, at acetic acid production',
 '62a5292     0.0164 [ Input] --: Natural gas, processed, at plant',
 '84623ff   3.23e-05 [ Input] --: Transport, pipeline, unspecified petroleum products',
 '2e320f5   0.000355 [ Input] --: Transport, barge, diesel powered',
 'a54e80a   0.000812 [ Input] --: Electricity, at grid, US, 2000',
 '3a3d3d4    0.00126 [ Input] --: Transport, barge, residual fuel oil powered',
 '05c3d96   0.000183 

In [36]:
F.frag('b8e').reference_entity = None

In [28]:
F.fragment_to_background(F.frag('094'))

LcFragment - Name: Transport, barge, residual fuel oil powered:
BG: ((B) ref) -<- 77a4145 -<- -*    Transport, barge, residual fuel oil powered [RNA]


In [ ]:
for f in ('')

In [33]:
F.frag('ba3c')._background=False

In [36]:
F.build_child_flows(F.frag('ba3'), background_children=True)

LcFragment - Name: Diesel, at refinery:


In [31]:
auto_terminate(F.frag('ba3c'))

(selecting only choice)
Picked: (USLCI) Transport, combination truck, diesel powered [Truck Transportation, General Freight Trucking] <- Transport, combination truck, diesel powered [RNA]


In [25]:
F.frag('15f1').terminate(None)

In [29]:
foreground(F.frag('28b5'))

(selecting only choice)
Picked: (USLCI) Acetic acid, at plant [Chemical Manufacturing, All Other Basic Organic Chemical Manufacturing] <- Acetic acid, at plant [RNA]
LcFragment - Name: Transport, combination truck, diesel powered:
BG: ((B) ref) -<- ba3c424 -<- ---:  Transport, combination truck, diesel powered
LcFragment - Name: CUTOFF Disposal, solid waste, unspecified, to sanitary landfill:
LcFragment - Name: Transport, train, diesel powered:
LcFragment - Name: Natural gas, processed, at plant:
LcFragment - Name: Transport, barge, diesel powered:
LcFragment - Name: Electricity, at grid, US, 2000:
BG: ((B) ref) -<- 756d536 -<- ---:  Electricity, at grid, US, 2000
LcFragment - Name: Electricity, at cogen, for natural gas turbine:
LcFragment - Name: Transport, pipeline, unspecified petroleum products:
LcFragment - Name: Methanol, at plant:
LcFragment - Name: Natural gas, combusted in industrial boiler:
LcFragment - Name: Transport, pipeline, natural gas:
LcFragment - Name: Transport, ba

In [19]:
auto_terminate(F.frag('491'))

(selecting only choice)
Picked: (USLCI) Natural gas, combusted in industrial boiler [Utilities, Steam and Air-Conditioning Supply] <- Natural gas, combusted in industrial boiler [RNA]
LcFragment - Name: Transport, combination truck, average fuel mix:
BG: ((B) ref) -<- 949b684 -<- ---:  Transport, combination truck, average fuel mix
LcFragment - Name: Transport, pipeline, natural gas:
LcFragment - Name: Transport, train, diesel powered:
LcFragment - Name: Natural gas, processed, at plant:
BG: ((B) ref) -<- 583eea1 -<- ---:  Natural gas, processed, at plant


In [9]:
def create_quantity(self):
    name = input('Enter quantity name: ')
    unit = ifinput('Enter reference unit: ', 'kg')
    comment = ifinput('Enter comment: ', '')
    q = LcQuantity.new(name, unit, Comment=comment)
    self[0].add(q)
    return q


In [10]:
def edit_flow(self):
    flow = pick_one(self._catalog[0].flows())
    ch = cyoa('Edit (P)roperties or (C)haracterizations? ', 'pc').lower()
    if ch == 'p':
        _edit_entity(flow)
    elif ch == 'c':
        edit_characterizations(flow)


def _edit_entity(entity):
    print('Select field to edit:')
    field = menu_list(*entity.keys())
    if field == -1 or field is None:
        return True
    new = ifinput('Enter new value for %s: ' % field, flow[field])
    if len(new) > 0:
        entity[field] = new
    else:
        print('Not updating.')

def edit_characterizations(flow):
    char = pick_one(cf for cf in flow.characterizations())
    val = float(ifinput('enter new characterization value: ', char.value ))
    char.value = val

In [13]:
def pick_fragment(background=False):
    """
    self is a foreground manager
    """
    frag = pick_list([f for f in F[0].fragments(background=background)])
    print('%s' % frag)
    return frag
    
    

In [14]:
def show_info(frag):
    frag.show()
    print('Exchange Values:')
    evs = list(frag._exchange_values.keys())
    evs.remove(0)
    evs.remove(1)
    print('%20.20s: %g' % ('Cached', frag.cached_ev))
    print('%20.20s: %g' % ('Observed', frag.observed_ev))
    for k in evs:
        print('%20.20s: %g' % (k, frag.exchange_value(k)))
    if frag._balance_flow:
        print('\nBalance flow: True (%s)' % frag.flow.reference_entity)
    else:
        print('\nBalance flow: False')    

def show_terminations(frag):
    print('Terminations: ')
    print('%20s: %s' % ('Scenario', 'Termination'))
    for k, v in frag._terminations.items():
        if v.descend:
            desc = '(agg)'
        else:
            desc = '     '
        print('%20.20s: %s %s %s' % (k, v, desc, v.term_node))

In [28]:
def _pick_scenario(scs):
    sc = _pick_list(scs, 'New scenario', 'Cancel')
    if sc == (None, None):
        return None
    if sc == (None, 1):
        scen = input('Enter scenario Name: ')
    elif sc == (None, 2):
        return None
    else:
        scen = scs[sc[0]]
    if scen == '0':
        scen = 0
    return scen

def _f_add_child(frag):
    return create_fragment(frag)

def _f_info(frag):
    _edit_entity(frag)
    
def _f_exch(frag):
    action = cyoa('Set (O)bserved exchange value, (S)cenario exchange value, (B)alance? ', 'osb')
    if action == 'o':
        frag.observed_ev = float(ifinput('Enter observed exchange value: ', frag.observed_ev))
    elif action == 's':
        scs = list(frag._exchange_values.keys())
        if frag.cached_ev != 1.0:
            scs.remove(0)
        scs.remove(1)
        sc = _pick_scenario(scs)
        if sc == None:
            return
        elif sc == 0:
            cev = frag.cached_ev
        elif sc in frag._exchange_values.keys():
            cev = frag.exchange_value(sc)
        else:
            cev = 1.0
        ev = float(ifinput('Enter exchange value: ', cev))
        frag.set_exchange_value(sc, ev)
    elif action == 'b':
        if frag._balance_flow:
            frag.unset_balance_flow()
        else:
            frag.set_balance_flow()

def _f_terms(frag):
    scs = list(frag.terminations())
    sc = _pick_scenario(scs)
    term = find_termination(frag)
    if term is None:
        ch = cyoa('No terminations found. Search for termination by (P)rocess or by (F)low? ', 'pf')
        
    else:
        add_termination(frag, term, scenario=sc)

def _f_select_child(frag):
    ch = pick_list([f for f in F.child_flows(frag)])
    edit_fragment(ch)
        
def edit_fragment(frag):
    print('%s' % frag)
    F._show_frag_children(frag)
    show_info(frag)
    show_terminations(frag)
    action = {'a': _f_add_child, 'i': _f_info, 'e': _f_exch, 't': _f_terms, 's': _f_select_child, 'q': None
             }[cyoa('Choices: (A)dd child, (I)nfo, (E)xchange values, (T)erminations; (S)elect child, (Q)uit: ', 'aeitsq', 'i')]
    if action is None:
        return
    action(frag)
    

In [21]:
F.show_fragments(background=None)

( ** ref) -<- de6bbb6 -<- ---:  Polyethylene terephthalate, resin, at plant


In [29]:
edit_fragment(frag)

( ** ref) -<- de6bbb6 -<- -*    Polyethylene terephthalate, resin, at plant
 (de6bbb6) -<- ebfc8dd -<- ---:  Bituminous coal, combusted in industrial boiler
 (de6bbb6) -<- a138003 -<- ---:  Diesel, combusted in industrial equipment
 (de6bbb6) -<- 2f5d5a7 -<- ---:  Paraxylene, at plant
 (de6bbb6) -<- b0adf5b -<- ---:  Residual fuel oil, combusted in industrial boiler
 (de6bbb6) -<- 06bd393 -<- ---:  Electricity, at grid, US, 2008
 (de6bbb6) -<- e91a6c4 -<- ---:  CUTOFF Ethylene glycol, at plant
 (de6bbb6) -<- 1cd07b2 -<- ---:  CUTOFF Disposal, solid waste, unspecified, to sanitary landfill
 (de6bbb6) -<- 314ca25 -<- ---:  Transport, barge, residual fuel oil powered
 (de6bbb6) -<- e599048 -<- ---:  Methanol, at plant
 (de6bbb6) -<- 34e3cff -<- ---:  Transport, train, diesel powered
 (de6bbb6) -<- bf4b217 -<- ---:  Acetic acid, at plant
 (de6bbb6) -<- cf302f7 -<- ---:  Transport, barge, diesel powered
 (de6bbb6) -<- 038c03f -<- ---:  Electricity, at cogen, for natural gas turbine
 (de6bbb

In [72]:
frag.term.__dict__

{'_cached_ev': 1.0,
 '_descend': True,
 '_parent': <lcatools.foreground.fragment_flows.LcFragment at 0x7feb09e84588>,
 '_process_ref': None,
 '_score_cache': {},
 'direction': 'Output',
 'term_flow': None}

In [64]:
frbc = F.frag('ebfc')

In [66]:
help(del)

SyntaxError: invalid syntax (<ipython-input-66-0bb27ba05bc1>, line 1)

In [63]:
F.draw_fragment(frag)

-<--*   de6bbb6 (      1) Polyethylene terephthalate, resin, at plant
    | -<----: b0adf5b (  0.027) Residual fuel oil, combusted in industrial boiler
    | -<----: 06bd393 (   0.56) Electricity, at grid, US, 2008
    | -<----: e91a6c4 (   0.32) CUTOFF Ethylene glycol, at plant
    | -<----: 1cd07b2 ( 0.0042) CUTOFF Disposal, solid waste, unspecified, to sanitary landfill
    | -<----: 314ca25 ( 0.0022) Transport, barge, residual fuel oil powered
    | -<----: 34e3cff (    1.6) Transport, train, diesel powered
    | -<----: cf302f7 (0.00065) Transport, barge, diesel powered
    | -<----: 98a4235 (0.00059) CUTOFF Disposal, solid waste, unspecified, to waste-to-energy
    | -<----: bf4b217 (  0.037) Acetic acid, at plant
    | -<----: 85c992f (8.8e-05) Transport, pipeline, natural gas
    | -<----: 038c03f (  0.051) Electricity, at cogen, for natural gas turbine
    | =>=---: 9cad754 (   0.14) Recovered energy
    | -<----: c6405b9 (0.00031) CUTOFF Disposal, solid waste, unspecified, to

In [58]:
auto_terminate(F.frag('2f5d'), index=8)

(selecting only choice)
Picked: (USLCI) Paraxylene, at plant [Chemical Manufacturing, Petrochemical Manufacturing] <- Paraxylene, at plant [RNA]


In [48]:
str(flow_ref)

'(USLCI) {f} Ethylene glycol, at plant  [Chemical Manufacturing, Petrochemical Manufacturing]'

In [51]:
x = find_termination(flow_ref, direction)

(selecting only choice)
Picked: (USLCI) Ethylene glycol, at plant  [Chemical Manufacturing, Petrochemical Manufacturing] <- Ethylene glycol, at plant  [RNA]


In [40]:
str(x.exchange)

'Output: {*} [1 [kg]] Polyethylene terephthalate, resin, at plant [Chemical Manufacturing, Petrochemical Manufacturing]'